# Showcase ROUNDING across MSAccess, Python, Matlab & C&#35;

## Linux dependencies for this notebook:
- *octave*: install it with:

      sudo apt install octave


- *dotnet-core*: 
  - Follow [these `apt`(*Debian, Ubutnu*) instructions](https://dotnet.microsoft.com/download/linux-package-manager/debian9/sdk-current).
    to install .NET SDK from MS repository:
  - Source `/etc/profile` (OR append `~/.dotnet/tools` in *PATH*, OR login & logout).
  - Install REPL for *C#*: 
  
        dotnet tool install -g dotnet-script
        

## INPUT numbers to be rounded
Interesting numbers taken from: https://stackoverflow.com/a/45424214/548792

In [1]:
a = [0.49999999999999994, 5000000000000001.0, -2.4, 2.4]
#a = [0.499999999999994, 500000000000001.0, -2.4, 2.4]  Access input
a

[0.49999999999999994, 5000000000000001.0, -2.4, 2.4]

## *MSAccess* behavior
While *Access* internally stores floats as `doubles` (8-bytes) <br>
**BUT...** the GUI only allows floats with 15 decimals, and not 17 needed to [represent IEEE 754 doubles](https://en.wikipedia.org/wiki/Double-precision_floating-point_format#IEEE_754_double-precision_binary_floating-point_format:_binary64)
(15 decimals are enough to be preserved when passed through an IEEE 754 double).


```
                Input             Out1               Out2             Out3
    0.499999999999994                0  0.999999999999994              0.5     ## Input had 15 decimals (instead of 17)!!
    500000000000001    500000000000001    500000000000002  500000000000001     ## Input had 15 digits (instead of 16)!!
    -2.4                            -2               -1.9             -2.4
    2.4                              2                2.9             2.4


WHERE:
    - Out1: int(Input + 0.5)
    - Out2: Input + 0.5
    - Out3: int(10 * Input + 0.5) / 10
```

In [2]:
# COPIED manually from `Out1`, above
r_access = [0, 500000000000001, -2, 2]

## Python floats

In [3]:
import math
%precision 17

'%.17f'

In [4]:
r_python = [math.floor(n + 0.5) for n in a]
r_python

[1, 5000000000000002, -2, 2]

## Numpy float64

In [5]:
import numpy as np

## Note that `%precision` magic also modifies numpy's precision.
np.set_printoptions(precision=17, floatmode='fixed', suppress=True)

In [6]:
aa = np.array(a)
r_numpy = np.floor(aa + 0.5)
r_numpy

array([ 1.00000000000000000e+00,  5.00000000000000200e+15,
       -2.00000000000000000e+00,  2.00000000000000000e+00])

## Python DECIMALS

In [7]:
from decimal import Decimal as D, ROUND_HALF_UP

def round_dec(n):
    decimals = D('1')
    n = D(n).quantize(decimals, rounding=ROUND_HALF_UP)
    return float(n)
r_decimals = [round_dec(n) for n in a]
r_decimals

[0.00000000000000000,
 5000000000000001.00000000000000000,
 -2.00000000000000000,
 2.00000000000000000]

## Matlab (Octave)

In [8]:
%%script octave --out r_octave
format long
a = [0.49999999999999994, 5000000000000001.0, -2.4, 2.4];
%a = [0.499999999999994, 500000000000001.0, -2.4, 2.4];  % Access input
disp(sprintf(repmat('%0.17g ', 1, length(a)), floor(a + 0.5)))

In [9]:
r_matlab = [float(n) for n in r_octave.strip().split(' ')]
r_matlab

[1.00000000000000000,
 5000000000000002.00000000000000000,
 -2.00000000000000000,
 2.00000000000000000]

## .Net Core C&#35;

In [10]:
import os

tmpfile = f"/tmp/rounding-{os.getpid()}.csx"

In [11]:
%%writefile $tmpfile
double [] a = {0.49999999999999994, 5000000000000001.0, -2.4, 2.4};
//double [] a = {0.499999999999994, 500000000000001.0, -2.4, 2.4};  // Access input
for(int i = 0; i < a.GetLength(0); i++)
    Console.WriteLine(Math.Floor(a[i] + 0.5).ToString("F"));

Writing /tmp/rounding-16121.csx


In [12]:
r_dotnet = !dotnet-script $tmpfile
!rm $tmpfile
r_dotnet = [float(n) for n in r_dotnet]
r_dotnet

[1.00000000000000000,
 5000000000000000.00000000000000000,
 -2.00000000000000000,
 2.00000000000000000]

# COMPARE

In [13]:
import pandas as pd

df = pd.DataFrame(np.array([r_access, r_python, r_numpy, r_decimals, r_matlab, r_dotnet]).T,
                  columns="access python numpy decimals matlab dotnet".split(),
                 index=a)
display(df)

,access,python,numpy,decimals,matlab,dotnet
5.000000e-01,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
5.000000e+15,5.000000e+14,5.000000e+15,5.000000e+15,5.000000e+15,5.000000e+15,5.000000e+15
-2.400000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00
2.400000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00


## Comments:
- All implementations but "decimals" fail to reproduce *access* on number 0.49999999999999994. <br>
  - *Access* works this way because it accepts only "decimals(15)" as input (not IEEE 754 doubles that require 17 decimals).
  - If same input as *access* is given to all impls (by uncommenting lines), then thay all agree.
- For all the rest numbers, all implementations work fine with `floor(n + 0.5)` function.